In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1,0"

In [2]:
from time import time
import math
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer as skTfidf
import cupy as cp           #use cupy array instead of numpy to speed up calculation by using GPU
import cudf as cf
from cuml.metrics.pairwise_distances import sparse_pairwise_distances
from cuml.feature_extraction.text import TfidfVectorizer as cuTfidf
from sklearn.metrics import pairwise_distances


import matplotlib.pyplot as plt
import sys

In [3]:
# path = '/home/test/Data/corpus.csv'
# path = '/Users/lsacy/data/corpus.csv'
path = './corpus.csv'
def load_frame(path_to_df=path, encoding='utf-16', filter_value = 100000):
    
    df= pd.read_csv(path_to_df, encoding=encoding, index_col='id')
    df.drop([df.columns[0]], inplace=True, axis=1)
    df.drop_duplicates(subset=['text'],inplace=True)
    
    exam_type_distribution = df.groupby(['exam_type'])['exam_type'].count()
    exam_type_distribution.sort_values(ascending=False)
    
    list_filtered = exam_type_distribution[exam_type_distribution > filter_value].index
    df_filtered = df[df['exam_type'].isin(list_filtered)]
    
    print(f'original dataframe shape: {df.shape}')
    print(f'df_filtered shape: {df_filtered.shape}')
    
    print(f'numbers of exam types before: {len(set(df["exam_type"]))}')
    print(f'types after filtering: {list_filtered}')
    print(f'number of exam types after filtering: {len(set(df_filtered["exam_type"]))}')

    return df_filtered, list_filtered

In [4]:
# define a load function to load each exam type
def load_exam_type(df, exam_types):
    for i in exam_types:
        print(f'exam type: {i}')
        dataframe = df[df['exam_type'] == i]    
        print(f'number of documents: {dataframe.shape}')
        yield dataframe

In [5]:
def batch_tfidf(sparseMatrix, size = 5000):
    for idx, item in enumerate(range(0, sparseMatrix.shape[0], size)):
        batch_sparseMatrix = sparseMatrix[item:item+size,:]
        print(f'batch shape: {batch_sparseMatrix.shape}, item: {item} - {item+size}')
        yield batch_sparseMatrix

In [6]:
def get_distances(upperbound, lower, batch_size=8000, filter=100000):

    df_filtered = load_frame(filter_value = filter)
    df_by_type = load_exam_type(df_filtered[0], df_filtered[1])
    results_dict = {}

    # loop through each exam type
    for i in range(0,len(df_filtered[1])):
        print(f'exam {i}/{len(df_filtered[1])}')
        dataframe = next(df_by_type)
        df_indices = dataframe.index.to_list()

        tfidf = cuTfidf().fit_transform(dataframe['text'])
        
        batch = batch_tfidf(tfidf, size=batch_size)
        total_number_batches = math.ceil(tfidf.shape[0]/batch_size)
        print(f'# of batches: {total_number_batches} | batch size: {batch_size}')
        
        # loop through batches of tfidf matrix row wise
        counter = 0
        for i in range(0, math.ceil(tfidf.shape[0]/batch_size)):
            print(f'batch {i+1}/{total_number_batches}')
            
            batch_sparse = next(batch)
            
            distance_batch = sparse_pairwise_distances(batch_sparse, tfidf, metric='euclidean') # distance matrix

            sort_by_distance(distance_batch, df_indices, upperbound, results_dict, lower)
                                         
            del distance_batch
            del batch_sparse
                                         
            counter += batch_size
            print(f'results dict length: {len(results_dict)}')
            print('')
            


        del tfidf
        del batch
        
                
    return results_dict

In [7]:
def sort_by_distance(distance_batch, df_indices, upperbound, results, lower):
    #distance_batch= distance_batch[0:100] #take a sample of 100
    found = 0
    for i, row in tqdm(enumerate(distance_batch)):
        sorted_array = cp.sort(row)
        arg_sorted = cp.argsort(row)

        candidates, distances = get_candidates(sorted_array, lower, arg_sorted, upperbound)


        if len(candidates) > 1:
            found += 1

            df_candidates = [df_indices[int(i)] for i in candidates]
            original_index = df_candidates[0]
            df_candidates = df_candidates[1:]
            
            save_results(distances, results, original_index, df_candidates)
            
            del sorted_array, original_index, df_candidates, distances, arg_sorted, candidates
            
    print(f'{found} matches found')

    return results

In [8]:
def save_results(distances, results, original_index, df_candidates):
    results[original_index] = (df_candidates, distances)
    return results

In [9]:
def get_candidates(sorted_array, lower, arg_sorted, upper):
    candidates = []
    distances = []
    for i, x in enumerate(sorted_array):
        if x > upper:
            break
        elif x < lower:
            continue
        else:
            candidates.append(arg_sorted[i])
            distances.append(float(x))
    return candidates, distances[1:]

In [10]:
test = get_distances(upperbound=0.6, lower=0, batch_size=6500, filter=10000)

original dataframe shape: (2586631, 4)
df_filtered shape: (1845401, 4)
numbers of exam types before: 985
types after filtering: Index(['ARCK', 'ARCW', 'ARKK', 'ARKW', 'ARRB1EBBEK', 'ARRBXEB', 'ARREXEBO9L',
       'ARREXEBO9R', 'ARREXEBOEL', 'ARREXEBOER', 'ARREXEBOGL', 'ARREXEBOGR',
       'ARREXEBOHL', 'ARREXEBOHR', 'ARREXEBOSL', 'ARREXEBOSR', 'ARREXEBUFL',
       'ARREXEBUFR', 'ARREXEBUGL', 'ARREXEBUGR', 'ARREXEBUHL', 'ARREXEBUHR',
       'ARREXEBUKL', 'ARREXEBUKR', 'ARRKOPG', 'ARRKXEBNNH', 'ARRT', 'ARRTRBS',
       'ARRWXEBBWS', 'ARRWXEBHWS', 'ARRWXEBLWS', 'ARSB', 'ARSBSNONIX',
       'ARSBSNOTXN', 'ARSEFKVUVX', 'ARSESNOUXX', 'ARSXSNOWGW', 'ARXXTLE'],
      dtype='object', name='exam_type')
number of exam types after filtering: 38
exam 0/38
exam type: ARCK
number of documents: (100888, 4)
# of batches: 16 | batch size: 6500
batch 1/16
batch shape: (6500, 66772), item: 0 - 6500


6500it [00:07, 912.99it/s] 


452 matches found
results dict length: 451

batch 2/16
batch shape: (6500, 66772), item: 6500 - 13000


6500it [00:06, 943.25it/s] 


503 matches found
results dict length: 953

batch 3/16
batch shape: (6500, 66772), item: 13000 - 19500


...

799 matches found
results dict length: 254719

batch 4/5
batch shape: (6500, 32446), item: 19500 - 26000


6500it [00:06, 1058.05it/s]


734 matches found
results dict length: 255430

batch 5/5
batch shape: (896, 32446), item: 26000 - 32500


896it [00:00, 1152.66it/s]

119 matches found
results dict length: 255544



In [ ]:
df, df_types = load_frame()

In [ ]:
df_type

In [ ]:
print(df.loc[110471]['text'])

In [ ]:
print(df.loc[4421195]['text'])

In [ ]:
print(df.loc[1309976]['text'])

In [11]:
import pickle
filename = 'distances_below_0.6'
outfile = open(filename, 'wb')



In [12]:
pickle.dump(test, outfile)
outfile.close()

In [13]:
infile = open(filename, 'rb')
new_dict = pickle.load(infile)
infile.close

<function BufferedReader.close>

In [ ]:

for idx, (i, j) in enumerate(test.items()):
    # loop through every 10th item in j[0]
    if idx ==0 or idx//5 == 0:
        print(f'# of candidates: {len(j[0])}\n')
        original_text = df.loc[i]['text']
        print(f'original text:\n{original_text}')
        for i, k in enumerate(j[0][::50]):

            text = df.loc[k]['text']

            print(f'\nVergleichstext mit distance: {j[1][i]} \n{text}')


            #print(dataframe.loc[k]['text'])

            print('')

In [ ]:
# deduplicate with saved results_dict

for i, j in test.items():
    items_to_delete = [i for i in j[0] if i not in deleted_items]
    print(i, items_to_delete)
    for ij in items_to_delete:
        deleted_items.append(ij)
        print(f'deleted item: {deleted_items}')
    if i not in deleted_items:
        print('True')
        df = df.drop([*items_to_delete])
        deleted_items.append(i)
